# Batch protein preparation

This notebook shows how to load a large amount of data and execute runs on them in parallel

In [1]:
import os
import asyncio
from glob import glob
from datetime import datetime
from pathlib import Path

from pdbtools import (
    pdb_fetch,
    pdb_delhetatm,
    pdb_selchain,
    pdb_rplresname,
    pdb_keepcoord,
    pdb_selresname,
)

import rush

### 0) Setup

In [2]:
# Set our token - ensure you have exported RUSH_TOKEN in your shell; or just
# replace the os.getenv with your token
TOKEN = os.getenv("RUSH_TOKEN")
URL = os.getenv("RUSH_URL")

In [3]:
# Define our project information
DESCRIPTION = "rush-py batch notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "batch-prep"]
WORK_DIR = Path.home() / "qdx" / "rush-py-batch-prep"

# Set our inputs
PROTEIN_PDB_FOLDER_PATH = WORK_DIR / "proteins"

TARGET = "NIX_SSH"

In [4]:
# |hide
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

Ensure your workdir exists

In [5]:
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(PROTEIN_PDB_FOLDER_PATH, exist_ok=True)

## Initialize our rush client and fetch available module paths

In [6]:
# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(
    access_token=TOKEN, url=URL, workspace=WORK_DIR, batch_tags=TAGS
)

In [7]:
# |hide 
client = await rush.build_provider_with_functions(
    access_token=TOKEN, url=URL, workspace=WORK_DIR, batch_tags=TAGS, restore_by_default=True
)
client.restore_by_default

True

In [8]:
# fetch datafiles
for pdb in ["1B39", "4QXI", "8FSU"]:
    complex = list(pdb_fetch.fetch_structure(pdb))
    protein = pdb_delhetatm.remove_hetatm(pdb_selchain.select_chain(complex, "A"))

    with open(PROTEIN_PDB_FOLDER_PATH / f"{pdb}_protein.pdb", "w") as f:
        for l in protein:
            f.write(str(l))

In [9]:
help(client.prepare_protein)

Help on function prepare_protein in module rush.provider:

async prepare_protein(*args: *tuple[RushObject[bytes]], target: Optional[Target] = 'NIX_SSH', resources: Optional[Resources] = {'storage': 138, 'storage_units': 'MB', 'gpus': 1}, tags: list[str] | None = None, restore: bool | None = False) -> tuple[RushObject[list[Conformer]], RushObject[bytes]]
    Prepare a PDB for downstream tasks: protonate, fill missing atoms, etc.
    
    Module version:  
    `github:talo/prepare_protein/83bed2ad1f01f495c94518717f9f5b1bd7fe855c#prepare_protein_tengu`
    
    QDX Type Description:
    
        input_pdb: @bytes
        ->
        output_qdxf: @[Conformer];
        output_pdb: @bytes
    
    :param input_pdb: An input protein as a file; one PDB file
    :return output_qdxf: An output protein a vec: one qdxf per model in pdb
    :return output_pdb: An output protein as a file: one PDB file



## For each ligand, start a gmx + gmx_mmpbsa run

In [10]:
proteins = map(lambda x: Path(x), glob(str(PROTEIN_PDB_FOLDER_PATH / "*.pdb")))

protein_outputs = []

for protein_path in proteins:
    print(protein_path)
    name = protein_path.stem
    (prepped_protein_qdxf, prepped_pdb) = await client.prepare_protein(
        protein_path,
        target=TARGET,
        tags=[name],
    )
    protein_outputs.append((name, prepped_protein_qdxf))

protein_outputs

/root/qdx/rush-py-batch-prep/proteins/4QXI_protein.pdb


NameError: name 'tags' is not defined

## Report progress
This will show the status of all of your runs

In [12]:
status = await client.status(group_by="path")
print(f"{'Module':<32} | {'Status':<32} | Count")
print("-" * 75)
for module, (status, path, count) in status.items():
    print(f"{path:<32} | {status:<32} | {count:>5}")

Module                           | Status                           | Count
---------------------------------------------------------------------------
prepare_protein                  | ModuleInstanceStatus.ADMITTED    |     3


## Download Results
This will retrieve results for your completed module_instances

In [13]:
await asyncio.gather(
    *[
        output[1].download(filename=f"protein_{output[0]}.qdxf.json")
        for output in protein_outputs
    ],
)

2024-02-08 07:46:44,664 - rush - INFO - Argument ccff05f1-d9ff-4add-81ad-bc35dd07ed21 is now ModuleInstanceStatus.ADMITTED
2024-02-08 07:46:44,698 - rush - INFO - Argument 63352ac2-7b06-4eae-a4af-78722b67c81b is now ModuleInstanceStatus.RUNNING
2024-02-08 07:46:44,799 - rush - INFO - Argument 825527aa-bad7-401f-846d-fd5dcc41bdbf is now ModuleInstanceStatus.AWAITING_UPLOAD
2024-02-08 07:46:49,120 - rush - INFO - Argument 63352ac2-7b06-4eae-a4af-78722b67c81b is now ModuleInstanceStatus.AWAITING_UPLOAD
2024-02-08 07:46:58,746 - rush - INFO - Argument ccff05f1-d9ff-4add-81ad-bc35dd07ed21 is now ModuleInstanceStatus.DISPATCHED
2024-02-08 07:46:59,820 - rush - INFO - Argument ccff05f1-d9ff-4add-81ad-bc35dd07ed21 is now ModuleInstanceStatus.RUNNING
2024-02-08 07:47:06,500 - rush - INFO - Argument ccff05f1-d9ff-4add-81ad-bc35dd07ed21 is now ModuleInstanceStatus.AWAITING_UPLOAD


[None, None, None]

## Check failures
This will retrieve failed runs in your workspace history

In [14]:
for instance_id, (status, name, count) in (await client.status()).items():
    if status.value == "FAILED":
        async for log_page in client.logs(instance_id, "stderr"):
            for log in log_page:
                print(log)